---
# Lab Task: Forward Bank Bills and Enhanced Yield Curve Bootstrap #
---

In this lab, you will extend the existing class structure to handle **forward-starting bank bills** and build a yield curve that can bootstrap using these forward instruments.

## Background: Forward Bank Bills

A **forward bank bill** is a contract made today to purchase a bank bill at a future date. Unlike a regular bank bill that starts today, a forward bank bill:
- Has a **start date** in the future
- Has a **maturity date** beyond the start date  
- Allows market participants to lock in rates for future periods

### Example

A "3-month bank bill starting in 6 months" (denoted as a 6x9 forward):
- You agree today to a price
- In 6 months (start date), you pay that price
- In 9 months (maturity), you receive the face value

### Cash Flows

A forward bank bill has two key dates:
1. **Start date**: The date when you pay the price (negative cash flow)
2. **Maturity date**: The date when you receive the face value (positive cash flow)

### Why Forward Bills Matter for Yield Curves

Forward bank bills provide information about **implied forward rates** between two future points in time. By including them in the reference portfolio, we can:
- Build a more complete yield curve
- Extract market expectations about future interest rates
- Improve interpolation between sparse maturity points

## Lab Tasks Overview

You will:
1. Create a `Forward_bank_bill` class that inherits from `Bank_bill`
2. Create a `YieldCurve_with_forward_bills` class that inherits from `YieldCurve` 
3. Update the bootstrap method to handle forward-starting instruments
4. Instantiate a portfolio with both spot and forward instruments
5. Build a yield curve and test it with a set of cash flows

---

### Step 0: Import Required Classes

Import the necessary classes and functions from the existing modules.

In [ ]:
from instrument_classes import Bank_bill, Bond, Portfolio, CashFlows
from curve_classes_and_functions import YieldCurve
import numpy as np
import pandas as pd

---

### Step 1: Create the Forward_bank_bill Class

Create a new class `Forward_bank_bill` that inherits from `Bank_bill` with the following specifications:

**Requirements:**
- Add a `start_date` parameter to the `__init__` method (in addition to the existing Bank_bill parameters)
- The `maturity` parameter should represent the total time from now to final maturity
- The `start_date` represents the time from now to when the bill starts
- Override the `set_cash_flows()` method to create cash flows at:
  - `start_date`: negative cash flow equal to `-price`
  - `maturity`: positive cash flow equal to `face_value`
- Add getter methods for `start_date` and the forward period (`maturity - start_date`)

**Hints:**
- Use `super().__init__()` to call the parent class constructor
- For a forward bill, the pricing relationship involves both the start and maturity dates
- The forward rate can be calculated from the price and the forward period

**Example usage (after implementation):**
```python
# A 3-month bill starting in 6 months (6x9 forward)
fwd_bill = Forward_bank_bill(face_value=100, start_date=0.5, maturity=0.75)
fwd_bill.set_ytm(0.04)  # 4% forward rate
fwd_bill.set_cash_flows()
```

In [ ]:
# Create your Forward_bank_bill class here:

class Forward_bank_bill(Bank_bill):
    
    # Your code here
    pass

---

### Step 2: Create the YieldCurve_with_forward_bills Class

Create a new class `YieldCurve_with_forward_bills` that inherits from `YieldCurve` and enhances the bootstrap method.

**Requirements:**

1. Override the `bootstrap()` method to handle forward bank bills
2. The method should:
   - Start by calling `self.add_zero_rate(0, 0)` to set the discount factor at time 0
   - Process regular (spot) bank bills first, in order of maturity
     - For each spot bill: extract the discount factor at its maturity
   - Process forward bank bills next, in order of their maturity (not start date)
     - For each forward bill: use the known discount factor at the start date and the bill's pricing to solve for the discount factor at the maturity date
   - Process bonds last, in order of maturity
     - For each bond: use all previously determined discount factors to value intermediate coupons, then solve for the discount factor at the bond's maturity

**Key Insight for Forward Bills:**

For a forward bill with:
- Price = P
- Face value = F  
- Start date = $t_1$
- Maturity = $t_2$

The present value equation is:
$$NPV = -P \cdot DF(t_1) + F \cdot DF(t_2) = 0$$

Therefore:
$$DF(t_2) = \frac{P}{F} \cdot DF(t_1)$$

**Hints:**
- You'll need to distinguish between regular bank bills and forward bank bills
- Use `isinstance(bill, Forward_bank_bill)` to check if an instrument is a forward bill
- You can access the portfolio's bank bills with `self.portfolio.get_bank_bills()`
- For forward bills, you'll need to call `get_discount_factor(start_date)` from the curve
- Regular bills are just forward bills with start_date = 0

In [ ]:
# Create your YieldCurve_with_forward_bills class here:

class YieldCurve_with_forward_bills(YieldCurve):
    
    def bootstrap(self):
        # Your code here
        pass

---

### Step 3: Create the Reference Portfolio

Now create a reference portfolio containing the following instruments:

**Required Instruments:**

1. **One regular 3-month bank bill** (0.25 years)
   - Face value: 100
   - YTM: 3.0%

2. **Three forward bank bills (all 3-month bills):**
   - **3x6 forward**: starts in 3 months (0.25 years), matures in 6 months (0.5 years)
     - Face value: 100
     - YTM: 3.2%
   - **6x9 forward**: starts in 6 months (0.5 years), matures in 9 months (0.75 years)
     - Face value: 100
     - YTM: 3.4%
   - **9x12 forward**: starts in 9 months (0.75 years), matures in 12 months (1.0 year)
     - Face value: 100
     - YTM: 3.6%

3. **One 1-year quarterly coupon bond** (1.0 year maturity)
   - Face value: 100
   - Coupon rate: 4.0% per annum
   - Frequency: 4 (quarterly)
   - YTM: 3.8%

4. **One 2-year annual coupon bond** (2.0 years maturity)
   - Face value: 100
   - Coupon rate: 4.5% per annum
   - Frequency: 1 (annual)
   - YTM: 4.2%

**Important Steps:**
- For each instrument, set the YTM using `.set_ytm()`
- For each instrument, call `.set_cash_flows()` to generate the cash flow schedule
- Create a `Portfolio` object and add all instruments in the correct order
- Remember: add bank bills (spot then forward) before bonds

In [ ]:
# Create the regular 3-month bank bill

# Your code here

In [ ]:
# Create the three forward bank bills (3x6, 6x9, 9x12)

# Your code here

In [ ]:
# Create the 1-year quarterly bond

# Your code here

In [ ]:
# Create the 2-year annual bond

# Your code here

In [ ]:
# Create a Portfolio and add all instruments

# Your code here

---

### Step 4: Build the Yield Curve

Now instantiate your `YieldCurve_with_forward_bills` class, set the constituent portfolio, and run the bootstrap method.

After bootstrapping, display the resulting discount factors and zero rates at the key maturities.

In [ ]:
# Create the yield curve instance

# Your code here

In [ ]:
# Display the curve results in a nice table
# Hint: Create a DataFrame with columns for Maturity, Discount Factor, and Zero Rate

# Your code here

---

### Step 5: Test the Yield Curve

Create a `CashFlows` object with the following cash flows and calculate its present value using the yield curve:

**Cash Flow Schedule:**
- 0.5 years: $25
- 1.0 year: $30
- 1.5 years: $35
- 2.0 years: $110

Use the `npv()` method from your yield curve to calculate the present value.

**Expected behavior:**
- The curve should interpolate discount factors at 1.5 years (which is not a reference portfolio maturity)
- You should see a reasonable present value (approximately in the range of $180-$195)

In [ ]:
# Create a CashFlows object and add the cash flows

# Your code here

In [ ]:
# Calculate and display the NPV

# Your code here

---

### Step 6: Analysis Questions

Answer the following questions based on your results:

1. **Forward Rate Interpretation**: Compare the YTMs of the regular 3-month bill (3.0%) with the forward bills (3.2%, 3.4%, 3.6%). What does this tell you about the market's expectation of future interest rates?

2. **Discount Factor Pattern**: Look at your discount factors. Do they decrease as maturity increases? Why is this expected?

3. **Zero Rate Curve Shape**: Plot or examine the zero rates from 0.25 to 2.0 years. Is the curve upward sloping, downward sloping, or flat? What does this imply about the term structure?

4. **Interpolation**: The cash flow at 1.5 years requires interpolation. What interpolation method does the ZeroCurve class use? (Hint: check the `get_discount_factor()` method)

5. **Bootstrap Order**: Why is it important to process the instruments in order of maturity when bootstrapping? What would go wrong if you tried to process the 2-year bond before all the shorter-maturity instruments?

**Your answers here:**

1. Forward Rate Interpretation:


2. Discount Factor Pattern:


3. Zero Rate Curve Shape:


4. Interpolation Method:


5. Bootstrap Order:


---

### Optional Extension: Visualize the Yield Curve

Create a plot showing:
1. The zero rate curve
2. The discount factor curve

Use matplotlib to create these visualizations.

In [ ]:
# Optional: Visualization code
import matplotlib.pyplot as plt

# Your code here

---

## Summary

In this lab, you have:
- ✅ Created a `Forward_bank_bill` class to represent forward-starting instruments
- ✅ Extended the `YieldCurve` class to handle forward bills in the bootstrap process
- ✅ Built a complete yield curve from a mixed portfolio of spot and forward instruments
- ✅ Tested the curve by valuing a set of cash flows
- ✅ Analyzed the term structure of interest rates

**Key Takeaways:**
- Forward instruments provide information about market expectations of future rates
- The bootstrap method must be carefully sequenced to build up the curve progressively
- Interpolation allows us to value cash flows at dates not in the reference portfolio
- The yield curve is a fundamental tool for pricing fixed-income securities

---